In [2]:
import json
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from functools import lru_cache

In [3]:
# Set your Spotify app credentials
SPOTIPY_CLIENT_ID = '330a4427250d4e1aab13cb14859ead7b'
SPOTIPY_CLIENT_SECRET = '07bb4e0d8f12486482224ed60bd7da7a'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'  # e.g., 'http://localhost:8080'

In [4]:
# Authenticate with Spotify using OAuth
sp = Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope="user-library-read,user-top-read"))

In [5]:
@lru_cache(maxsize=1)
def get_saved_tracks():
    saved_tracks = []
    tracks = sp.current_user_saved_tracks()
    while tracks:
        for i, item in enumerate(tracks['items']):
           saved_tracks.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return saved_tracks


In [7]:
saved_tracks = get_saved_tracks()

In [12]:
for track in saved_tracks:
    print(track['track']['name'], end="\n")

Space Bound
The Search
Burn the House Down
One Thing Right
Beautiful
Rap God
Glorious (feat. Skylar Grey)
Falling
PANDEMONIUM
HAPPY
Time
When I'm Gone
River (feat. Ed Sheeran)
Bye Bye Bye - From Deadpool and Wolverine Soundtrack
Feel This Moment (feat. Christina Aguilera)
This Love
Hotel California - 2013 Remaster
Dream On
I Was Made For Lovin' You
גיבורי על
Strangelove
Leave the City
Time of Our Lives
House of Memories
All My Loving - Remastered 2009
Firestone
Deep Down (feat. Never Dull)
Do It Right
Wind Of Change
Black Friday (pretty like the sun)
Uptown Funk (feat. Bruno Mars)
Walk Of Life
Summer
BURN IT DOWN
Flashdance...What a Feeling - Radio Edit
Never Thought That I Could Die
Push Up - Main Edit
Austin (Boots Stop Workin')
Billion Streams Freestyle
Poison
Alone
You Make Me
Titanium (feat. Sia)
Light It Up (Remix)
Where Them Girls At (feat. Nicki Minaj & Flo Rida)
Play Hard (feat. Ne-Yo & Akon)
Hall of Fame (feat. will.i.am)
Where Are You Now
Call on Me - Radio Mix
Just Can’t Ge

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [50]:
for item in top_tracks['items']:
    print(json.dumps(item))

{"album": {"album_type": "ALBUM", "artists": [{"external_urls": {"spotify": "https://open.spotify.com/artist/6fOMl44jA4Sp5b9PpYCkzz"}, "href": "https://api.spotify.com/v1/artists/6fOMl44jA4Sp5b9PpYCkzz", "id": "6fOMl44jA4Sp5b9PpYCkzz", "name": "NF", "type": "artist", "uri": "spotify:artist:6fOMl44jA4Sp5b9PpYCkzz"}], "available_markets": ["AR", "AU", "AT", "BE", "BO", "BR", "BG", "CA", "CL", "CO", "CR", "CY", "CZ", "DK", "DO", "DE", "EC", "EE", "SV", "FI", "FR", "GR", "GT", "HN", "HK", "HU", "IS", "IE", "IT", "LV", "LT", "LU", "MY", "MT", "MX", "NL", "NZ", "NI", "NO", "PA", "PY", "PE", "PH", "PL", "PT", "SG", "SK", "ES", "SE", "CH", "TW", "TR", "UY", "US", "GB", "AD", "LI", "MC", "ID", "JP", "TH", "VN", "RO", "IL", "ZA", "SA", "AE", "BH", "QA", "OM", "KW", "EG", "MA", "DZ", "TN", "LB", "JO", "PS", "IN", "KZ", "MD", "UA", "AL", "BA", "HR", "ME", "MK", "RS", "SI", "KR", "BD", "PK", "LK", "GH", "KE", "NG", "TZ", "UG", "AG", "AM", "BS", "BB", "BZ", "BT", "BW", "BF", "CV", "CW", "DM", "FJ", 